In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
!pip install datasets
!pip install -U bitsandbytes
!pip install peft

In [ ]:
import torch
import io
import pickle
import pandas as pd
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
from transformers import AutoModelForCausalLM, BitsAndBytesConfig


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [ ]:
import torch
from peft import LoraConfig
from transformers import AutoProcessor, BitsAndBytesConfig, Idefics2ForConditionalGeneration

DEVICE = "cuda:0"
USE_LORA = False
USE_QLORA = True

processor = AutoProcessor.from_pretrained(
    "HuggingFaceM4/idefics2-8b",
    do_image_splitting=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Chat templates should be in a 'chat_template.jinja' file but found key='chat_template' in the processor's config. Make sure to move your template to its own file.


In [ ]:
import torch
from transformers import BitsAndBytesConfig, AutoProcessor, Idefics2ForConditionalGeneration
from peft import AdaLoraConfig, get_peft_model
from peft import LoraConfig, AdaLoraModel, AdaLoraConfig
# Define AdaLoRA config
if USE_QLORA or USE_LORA:
    lora_config = AdaLoraConfig(
        init_r=8,                # Initial rank
        target_r=4,              # Target rank after adaptation
        beta1=0.85,              # Controls rank adjustment speed
        beta2=0.85,              # Second momentum parameter
        tinit=200,               # Iterations before rank adjustment starts
        tfinal=1000,             # Final iteration for adjustment
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules='.*(text_model|modality_projection|perceiver_resampler).*(down_proj|gate_proj|up_proj|k_proj|q_proj|v_proj|o_proj).*$',
        use_dora=False if USE_QLORA else True,
        init_lora_weights="gaussian"
    )

    # QLoRA compatibility
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    ) if USE_QLORA else None

    # Load model with quantization
    model = Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/idefics2-8b",
        torch_dtype=torch.float16,
        quantization_config=bnb_config,
    )

    # Add AdaLoRA adapter
    model = get_peft_model(model, lora_config)
else:
    # Without LoRA/QLoRA
    model = Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/idefics2-8b",
        torch_dtype=torch.float16,
        _attn_implementation="flash_attention_2",
    ).to(DEVICE)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("flaviagiammarino/vqa-rad")

# Split the dataset: 80% for training, 20% for validation
train_test_split = dataset['train'].train_test_split(test_size=0.2, seed=42)

# Rename the splits
train_dataset = train_test_split['train']
valid_dataset = train_test_split['test']
datasets = {
    'train': train_dataset,
    'valid': valid_dataset
}

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(valid_dataset)}")

Train size: 1434
Validation size: 359


In [ ]:
from datasets import load_dataset
from torch.utils.data import Dataset
from PIL import Image
import random

from datasets import load_dataset
from torch.utils.data import Dataset
from PIL import Image
import random

class VQADataset(Dataset):
    def __init__(self, hf_dataset, processor):
        self.dataset = hf_dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        question = self.dataset[idx]['question']
        answer = self.dataset[idx]['answer']
        image = self.dataset[idx]['image']  # This is a PIL image

        return {
            "image": image,  # Directly use the PIL image
            "query": {"en": question},
            "answers": [answer]
        }

# Define the MyDataCollator class
class MyDataCollator:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, examples):
        texts = []
        images = []
        for example in examples:
            image = example["image"]  # Directly use the PIL image
            question = example["query"]['en']
            answer = random.choice(example["answers"])

            # Create messages as before
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Answer briefly."},
                        {"type": "image"},
                        {"type": "text", "text": question}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": answer}
                    ]
                }
            ]
            text = self.processor.apply_chat_template(messages, add_generation_prompt=False)
            texts.append(text.strip())
            images.append(image)

        # Process texts and images using the processor
        batch = self.processor(text=texts, images=images, return_tensors="pt", padding=True)

        # Create labels for training
        labels = batch["input_ids"].clone()
        labels[labels == self.processor.tokenizer.pad_token_id] = -100  # Ignore padding in loss calculation
        batch["labels"] = labels

        return batch


train_dataset = VQADataset(hf_dataset=datasets['train'], processor=processor)
test_dataset = VQADataset(hf_dataset=dataset['test'], processor=processor)
valid_dataset = VQADataset(hf_dataset=datasets['valid'], processor=processor)

data_collator = MyDataCollator(processor=processor)


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback


training_args = TrainingArguments(
    output_dir="IDEFICS2",
    learning_rate=2e-4,
    fp16=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    dataloader_pin_memory=False,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=25,  # Increased evaluation frequency
    save_steps=25,
    max_steps=100,
    logging_steps=5,
    remove_unused_columns=False,
    push_to_hub=False,
    label_names=["labels"],
    report_to="none",
    optim="paged_adamw_8bit",
    weight_decay=0.05,
    # Add early stopping callback
    # load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop if no improvement for 3 evals
)

In [ ]:
trainer.train()


Step,Training Loss,Validation Loss
25,14.662200,13.953479


Step,Training Loss,Validation Loss
25,14.662200,13.953479
50,6.980600,5.573436
75,1.205800,1.072294
100,0.697800,0.667984


TrainOutput(global_step=100, training_loss=7.279871044158935, metrics={'train_runtime': 2861.4946, 'train_samples_per_second': 0.28, 'train_steps_per_second': 0.035, 'total_flos': 3632633149386528.0, 'train_loss': 7.279871044158935, 'epoch': 0.5578800557880056})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,LlavaProcessor

# Define save path
save_directory = "/content/drive/MyDrive/VQA13Feb/models/AdaIdeficsFinal"


from transformers import AutoConfig

# config = AutoConfig.from_pretrained("HuggingFaceM4/Idefics3-8B-Llama3")  # Use the base model
model.config.save_pretrained(save_directory)
model.save_pretrained(save_directory)
# processor = LlavaProcessor.from_pretrained("HuggingFaceM4/Idefics3-8B-Llama3")
processor.save_pretrained(save_directory)

print(f"✅ Model saved at: {save_directory}")

print(f"✅ Model saved at: {save_directory}")


✅ Model saved at: /content/drive/MyDrive/VQA13Feb/models/AdaIdeficsFinal
✅ Model saved at: /content/drive/MyDrive/VQA13Feb/models/AdaIdeficsFinal


In [ ]:
from accelerate.utils import release_memory
# ...

release_memory(model)

[None]

In [ ]:
test_results = trainer.evaluate(test_dataset)

# Print the evaluation results
print(test_results)

{'eval_loss': 0.6864265203475952, 'eval_runtime': 535.0387, 'eval_samples_per_second': 0.843, 'eval_steps_per_second': 0.843, 'epoch': 0.5578800557880056}


In [ ]:
slake = load_dataset("mdwiratathya/SLAKE-vqa-english")


In [ ]:
slaketest = VQADataset(hf_dataset=slake['test'], processor=processor)


In [ ]:
test_results_Slake = trainer.evaluate(slaketest)

print(test_results_Slake)

{'eval_loss': 0.9643982648849487, 'eval_runtime': 1166.8499, 'eval_samples_per_second': 0.909, 'eval_steps_per_second': 0.909, 'epoch': 0.5578800557880056}


In [ ]:
t_results = trainer.evaluate()

print(t_results)

{'eval_loss': 0.6679843664169312, 'eval_runtime': 397.9107, 'eval_samples_per_second': 0.902, 'eval_steps_per_second': 0.902, 'epoch': 0.5578800557880056}


In [ ]:
from huggingface_hub import HfApi

# Set up the model and tokenizer save path
save_directory = "/content/drive/MyDrive/VQA13Feb/models/AdaIdeficsFinal"
repo_name = "AdaIdeficsFinal20F"  # Specify the name of your model repository
user_name = "Dtarget"  # Your Hugging Face username

# Initialize Hugging Face API
api = HfApi()

# Create the repository as private
api.create_repo(repo_id=f"{user_name}/{repo_name}", private=True, repo_type="model")

# Upload model to Hugging Face
api.upload_folder(
    folder_path=save_directory,
    repo_id=f"{user_name}/{repo_name}",
    path_in_repo=""  # Upload to root of the repository
)

print(f"✅ Model uploaded to Hugging Face privately: https://huggingface.co/{user_name}/{repo_name}")


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/93.4M [00:00<?, ?B/s]

✅ Model uploaded to Hugging Face privately: https://huggingface.co/Dtarget/AdaIdeficsFinal20F
